In [4]:
import pandas as pd
import numpy as np
from numpy import random
from scipy.stats import norm
skaters = pd.read_csv(f'cleaned/master_skaters.csv')
skaters['exGF'] = skaters['TOI']*0.032+0.055
skaters['exGA'] = skaters['TOI']*0.032+0.055
skaters['eGF'] = skaters['TOI']*0.032+0.055
skaters['eGA'] = skaters['TOI']*0.032+0.055
skaters['GFAA'] = skaters['GF'] - skaters['eGF']
skaters['GAAA'] =  skaters['eGA'] - skaters['GA']
skaters['xGFAA'] = skaters['xGF'] - skaters['exGF']
skaters['xGAAA'] =  skaters['exGA'] - skaters['xGA']
skaters['Production'] = (0.75*skaters['Goals']) + (0.7*skaters['First Assists']) + (0.55*skaters['Second Assists']) + (0.5*skaters['ixG']) + (0.05*skaters['Shots Blocked']) 
skaters['Misc'] = (0.15*(skaters['Penalties Drawn'] - skaters['Total Penalties'])) + (0.01*(skaters['Faceoffs Won'] - skaters['Faceoffs Lost']))
skaters['Offense'] = np.where(skaters['Position'] == 'D', 
((0.425*(skaters['GF'] + skaters['GFAA'])) + (1.7*(skaters['xGF'] + skaters['xGFAA'])))/2.125 - 0.75,
(((0.625*(skaters['GF'] + skaters['GFAA'])) + (0.625*(skaters['xGF'] + skaters['xGFAA'])))/2) - 0.4)
skaters['Defense'] = np.where(skaters['Position'] == 'D', 
(((0.575*(-skaters['GA'] + skaters['GAAA'])) + (2.3*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 1.05, 
(((0.4375*(-skaters['GA'] + skaters['GAAA'])) + (1.7*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 0.6)
skaters['Score'] = skaters['Production'] + skaters['Misc'] + skaters['Offense'] + skaters['Defense']
skaters = skaters.loc[:, ~skaters.columns.str.contains('^Unnamed')]
births = pd.read_csv('birthdates.csv')
merged = skaters.merge(births[['Player', 'Season', 'dob']], how='left', on=['Player', 'Season'])
merged.drop_duplicates(subset=['Player', 'Team', 'Date', 'Opposition', 'Season'], inplace=True)
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team == 'New York Islanders')),'1999-07-26', merged['dob'])
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team != 'New York Islanders')),'1996-02-17', merged['dob'])
merged[["Date", "dob"]] = merged[["Date", "dob"]].apply(pd.to_datetime)
merged['Age'] = (merged["Date"] - merged["dob"]).dt.days/365
merged['RoundAge'] = np.floor(merged["Age"])


In [506]:
merged[merged.Season == 20222023].Team.unique()

array(['Nashville Predators', 'San Jose Sharks', 'New York Rangers',
       'Tampa Bay Lightning', 'Los Angeles Kings', 'Vegas Golden Knights',
       'Washington Capitals', 'Boston Bruins', 'Carolina Hurricanes',
       'Columbus Blue Jackets', 'Montreal Canadiens',
       'Toronto Maple Leafs', 'Colorado Avalanche', 'Chicago Blackhawks',
       'Anaheim Ducks', 'Seattle Kraken', 'Edmonton Oilers',
       'Vancouver Canucks', 'Pittsburgh Penguins', 'Arizona Coyotes',
       'Philadelphia Flyers', 'New Jersey Devils', 'Buffalo Sabres',
       'Ottawa Senators', 'New York Islanders', 'Florida Panthers',
       'Dallas Stars', 'Minnesota Wild', 'Calgary Flames',
       'Detroit Red Wings', 'Winnipeg Jets', 'St Louis Blues'],
      dtype=object)

In [327]:
agescores = merged.groupby(['Player', 'Season', 'RoundAge']).mean()[['Score']]
agescores['GP'] = merged.groupby(['Player', 'Season', 'RoundAge']).count()[['Date']]
agescores['TotalGP'] = merged.groupby(['Player', 'Season']).count()[['Date']]
agescores['SeasonPCT'] = agescores['GP']/agescores['TotalGP']
agescores

C:\Users\jetso\AppData\Local\Temp\ipykernel_5216\3678090357.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Score  GP  TotalGP  SeasonPCT
Player        Season   RoundAge                                  
A.J. Greer    20162017 19.0      0.393581   5        5   1.000000
              20172018 20.0     -0.108004   6       17   0.352941
                       21.0      0.309990  11       17   0.647059
              20182019 21.0      0.293076   1       15   0.066667
                       22.0      0.128682  14       15   0.933333
...                                   ...  ..      ...        ...
Zenon Konopka 20112012 30.0      0.233344  29       55   0.527273
                       31.0     -0.083532  26       55   0.472727
              20122013 32.0     -0.006794  36       36   1.000000
              20132014 32.0      0.161267  35       58   0.603448
                       33.0      0.060632  23       58   0.396552

[19433 rows x 4 columns]

In [400]:
final = pd.DataFrame()
base = merged.groupby(['Player', 'Season']).mean()
final['Player'] = [i[0] for i in base.index]
final['Season'] = [i[1] for i in base.index]
final['Score'] = [i for i in base.Score]

for age in range(18, 50):
    final['O'+str(age)] = 0

for idx in range(0, len(agescores)):
    final['O' + str(int(agescores.index[idx][2]))] = np.where(((final.Player == agescores.index[idx][0]) & (final.Season == agescores.index[idx][1])), agescores.SeasonPCT[idx], final['O' + str(int(agescores.index[idx][2]))]) 
final['Entry'] = ((final.Season.shift() - final.Season) != -10001.0) | (final.Player.shift() != final.Player)
final['Exit'] = (((final.Season.shift(-1) - final.Season) != 10001.0) | (final.Player.shift(-1) != final.Player)) & (final.Season != 20222023.0)

final.to_csv('age_encodings.csv')

C:\Users\jetso\AppData\Local\Temp\ipykernel_5216\2747084165.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [498]:


ages = pd.read_csv('d_age_values.csv')
for k in range(0, len(ages)):

    mean = ages['Scored'].to_list()[k]

    scale = ages['Scorex'].to_list()[k]

    n = ages['Date'].to_list()[k]



    hs = []
    trials = []
    for j in range(0, 1000):
        for i in range(0, 967-n):

            thresh = 0
            x = 1
            while x > thresh:
                thresh = norm.ppf(0.75, loc=mean, scale=scale)
                x = random.normal(loc=mean, scale=scale)
            
            hs.append(x)
        trials.append(sum(hs) / len(hs))
    print(ages['RoundAge'].to_list()[k], sum(trials) / len(trials))

18.0 -0.3674098693875436
19.0 -0.2399717752287932
20.0 -0.23098132237841504
21.0 -0.239005513850067
22.0 -0.17125302714394958
23.0 -0.18151094818778493
24.0 -0.18529745824194413
25.0 -0.1737950772014055
26.0 -0.19858830113643483
27.0 -0.19957843392275681
28.0 -0.21023159307638203
29.0 -0.1973486450029603
30.0 -0.19278783307916592
31.0 -0.19100460828196794
32.0 -0.21681680636085138
33.0 -0.21197492083097202
34.0 -0.2053412508743638
35.0 -0.3022356419443061
36.0 -0.24644614778162488
37.0 -0.17560101000522482
38.0 -0.1582611080836638
39.0 -0.13015765265289553
40.0 -0.04097368669915049
41.0 0.23310371369821686
42.0 -0.08329229541482776


KeyboardInterrupt: 

In [37]:
df['Potential'].std()

nan

In [64]:
players = merged[merged.Season == 20222023].Player.unique()
names = []
ovrs = []
pots = []
teams = []
gps= []
pos = []
ages = []
raw = []
hots = []
for player in players:
    df = pd.read_csv(f'projections/20222023/{player.replace(" ", "_")}.csv')
    thresh = max(0, norm.ppf(0.95, loc=df['Potential'].mean(), scale=df['Potential'].std()))
    recent = df['Potential'].to_list()[-1]
    names.append(player)
    teams.append(df['Team'].to_list()[-1])
    raw.append(df['XGB'].to_list()[-1])
    ovrs.append(min(99, round(100*(df['XGB'].to_list()[-1]+5)/(7.07))))
    blend = min(2, (thresh+recent)/2)
    pots.append(max(round(100*(df['XGB'].to_list()[-1]+5)/(7.07)), round(100*(blend+5)/(7.07))))
    gps.append(len(df))
    pos.append(df['Position'].to_list()[-1])
    ages.append(round(df['Age'].to_list()[-1], 1))
    hots.append(min(99, round(100*(df['Kalman'].to_list()[-1]+5)/(7.07))))

ratings = pd.DataFrame()

ratings['Player'] = names
ratings['Age'] = ages

ratings['Position'] = pos

ratings['Team'] = teams
ratings['Raw'] = raw
ratings['Overall'] = ovrs
ratings['Potential'] = pots
ratings['Long Term Overall'] = hots
ratings['Hot Index'] = ratings['Long Term Overall'] - ratings['Overall'] 
ratings.to_csv('pages/ratings/data/ratings23.csv', index=False)
ratings.sort_values(by='Hot Index', ascending=False).head(20)

c:\Users\jetso\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\jetso\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
c:\Users\jetso\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\jetso\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
c:\Users\jetso\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a

,Player,Age,Position,Team,Raw,Overall,Potential,Long Term Overall,Hot Index
149,Brent Burns,38.1,D,Carolina Hurricanes,0.983324,85,85,91,6
346,Clayton Keller,24.7,R,Arizona Coyotes,0.684028,80,81,86,6
344,Barrett Hayton,22.8,C,Arizona Coyotes,0.383700,76,76,82,6
180,Vladislav Gavrikov,27.3,D,Los Angeles Kings,-0.014289,71,73,76,5
138,Jake DeBrusk,26.4,L,Boston Bruins,0.937570,84,84,89,5
305,Ryan Nugent-Hopkins,30.0,C,Edmonton Oilers,0.921356,84,84,89,5
455,Brandon Montour,29.0,D,Florida Panthers,0.718476,81,81,86,5
382,Damon Severson,28.6,D,New Jersey Devils,0.699063,81,81,86,5
525,Tyler Toffoli,30.9,R,Calgary Flames,0.860085,83,83,88,5
78,Anze Kopitar,35.6,C,Los Angeles Kings,0.878937,83,85,88,5


In [78]:
ratings.sort_values(by='Hot Index', ascending=False)[(ratings.Age < 35)].head(30)
ratings[ratings.Team == 'Philadelphia Flyers']

C:\Users\jetso\AppData\Local\Temp\ipykernel_24348\1232001838.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ratings.sort_values(by='Hot Index', ascending=False)[(ratings.Age < 35)].head(30)


,Player,Age,Position,Team,Raw,Overall,Potential,Long Term Overall,Hot Index
232,Lukas Sedlak,29.8,C,Philadelphia Flyers,0.525783,78,78,75,-3
362,Egor Zamula,NaN,D,Philadelphia Flyers,0.180327,73,74,73,0
363,Hayden Hodgson,26.6,R,Philadelphia Flyers,0.040223,71,72,71,0
364,Ivan Provorov,26.2,D,Philadelphia Flyers,0.124511,72,76,71,-1
365,James van Riemsdyk,33.9,L,Philadelphia Flyers,0.541178,78,80,77,-1
366,Joel Farabee,23.1,L,Philadelphia Flyers,0.297576,75,79,74,-1
367,Justin Braun,36.1,D,Philadelphia Flyers,0.110568,72,77,72,0
368,Kevin Hayes,30.9,C,Philadelphia Flyers,0.485472,78,79,76,-2
369,Morgan Frost,23.9,C,Philadelphia Flyers,0.259754,74,75,76,2
370,Nick Seeler,29.8,D,Philadelphia Flyers,0.246189,74,76,76,2


In [4]:
import plotly.graph_objects as go
import numpy as np
from moepy import lowess
from collections import namedtuple
import pickle
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor
from scipy.stats import norm
from datetime import datetime
start_dates = {season: datetime.strptime((str(season)[:4] + '-10-04'), '%Y-%m-%d') for season in skaters.groupby(['Season']).count().index}
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: f'𝒩(μ={s[0]:.3f}, 𝜎²={s[1]:.3f})'
def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain

    x = x + K*y      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)

def get_projections(player, time_type):

    df = merged[merged.Player == player]
    df = df.sort_values(by='Date')
    df['Career Game'] = range(1, len(df) + 1)
    df['Adjusted'] = df['Score']*(50/1.55) + 50
    df[["Date", "dob"]] = df[["Date", "dob"]].apply(pd.to_datetime)
    projections = [0]
    scores = scores = np.concatenate((np.zeros(10),  df['Score'].to_list()))

    for i in range(-9, len(df)):
        curr = 0
        total = 0
        start = 1
        j = i-1
        while start > 0.05 and j >= -10:
            if j >= 0:

                weight = 0.99968135**(df['Date'].to_list()[i] - df['Date'].to_list()[j]).days
            else:
                weight = 0.99968135**(-1*j)
            curr += weight * scores[j+10]
            total += weight
            start = weight/total
            j -= 1

        projections.append(1.35*(curr/total))            

        

    df['Projection'] = projections[10:]


    sensor_var = 1.87
    process_var = 0.035
    process_model = gaussian(0., process_var)
    pos = gaussian(0., 1.)
    N = len(df)
    zs = df['Score'].to_list()
    ps = []
    for i in range(N):
        prior = predict(pos, process_model)    
        pos = update(prior, gaussian(zs[i], sensor_var))
        ps.append(pos.mean)
    df['Kalman'] = ps

    # load the dataset
    series = df[['Projection', 'Kalman']]
    values = series.values

    # fit model
    model = pickle.load(open('finalized_model.sav', 'rb'))
    # construct an input for a new preduction
    preds = df['Kalman'].to_list()[:82]
    for i in range(82, len(df)):
        row = values[i-6:i].flatten()
    # make a one-step prediction
        yhat = model.predict(asarray([row]))
        preds.append(yhat[0])
    df['XGB'] = preds

    ages = range(18, 48)
    A = [-0.14396,-0.05957,0.021176,0.080782,0.144998,0.176731,
            0.181012,0.179595,0.162485,0.137085,0.114395,0.094328,
            0.071203,0.043198,0.013411,0.002706,-0.01087,-0.030415,
            -0.04996,-0.03732,-0.06303,-0.03521,-0.038033904,-0.033721952,
            -0.02941,-0.04036,-0.05131,-0.08806,-0.08892,-0.08978]
    poly  = np.polyfit(ages, A, 4)
    for i in range(1, len(preds)):
        prev_age = df['Age'].to_list()[i-1]
        prev_expected = np.polyval(poly, prev_age)
        curr_expected = np.polyval(poly, df['Age'].to_list()[i])
        multiplier = curr_expected/prev_expected
        if preds[i] < 0:
            preds[i] = preds[i]/multiplier
        else:
            preds[i] = preds[i] * multiplier



    for i in range(0, len(df)):
        season = df['Season'].to_list()[i]
        since_start = (df['Date'].to_list()[i] - start_dates[season]).days
        season_adjust = (-0.00109 * since_start  + 1.1)
        if df['Venue'].to_list()[i] == 'Home':
            preds[i] = preds[i]*1.35*season_adjust*1.2
        else:
            preds[i] = preds[i]*1.35*season_adjust*0.8


        prev = merged[(merged.Season == season) & (merged.Date < df['Date'].to_list()[i])]
        opps = prev.groupby(['Opposition']).mean(numeric_only=True)[['Score']]
        opps['Multiplier'] = (opps['Score']+5)/5.438829866884756
        mult = opps[opps.index == (df['Opposition'].to_list()[i])]['Multiplier']
        if len(mult) > 0:
            mult = mult[0]
        else:
            mult = 1
        preds[i] = preds[i] * mult

    df['adjXGB'] = preds
      
    # Model fitting
    frac = 0.35
    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, num_fits=20, robust_iters=3)
    df['XGB_Lowess'] = df_quantiles[0.5].to_list()

    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, num_fits=20, robust_iters=3)
    df['adjXGB_Lowess'] = df_quantiles[0.5].to_list()
    pots = []
    mults = []
    for i in range(0, len(df)):
        curr_age = df['Age'].to_list()[i]
        if curr_age <= 24.26795064:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = (0.37233423)/(curr_expected+0.2)
            mults.append(multiplier)
            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)

        elif curr_age <= 38.62152643:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
        elif curr_age <= 42.25458411:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = 0.24220954/(curr_expected+0.2)
            mults.append(multiplier)

            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)        
        else:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
    df['multiplier'] = mults
    df['Potential'] = pots
    df.reset_index(inplace=True, drop=True)
    df_quantiles.reset_index(inplace=True, drop=True)
    return df, df_quantiles



In [24]:
players = list(merged[merged.Season == 20222023].Player.unique())
players.sort()
players[450:]

['Justin Barron',
 'Justin Braun',
 'Justin Danforth',
 'Justin Faulk',
 'Justin Holl',
 'Justin Kirkland',
 'Justin Schultz',
 'Juuso Parssinen',
 'Juuso Valimaki',
 "K'Andre Miller",
 'Kaapo Kakko',
 'Kaedan Korczak',
 'Kaiden Guhle',
 'Kailer Yamamoto',
 'Kale Clague',
 'Karson Kuhlman',
 'Kasperi Kapanen',
 'Keegan Kolesar',
 'Kent Johnson',
 'Kevin Bahl',
 'Kevin Fiala',
 'Kevin Gravel',
 'Kevin Hayes',
 'Kevin Labanc',
 'Kevin Rooney',
 'Kevin Shattenkirk',
 'Kevin Stenlund',
 'Kiefer Sherwood',
 'Kieffer Bellows',
 'Kirby Dach',
 'Kirill Kaprizov',
 'Kirill Marchenko',
 'Klim Kostin',
 'Kris Letang',
 'Kristian Reichel',
 'Kurtis MacDermid',
 'Kyle Burroughs',
 'Kyle Capobianco',
 'Kyle Clifford',
 'Kyle Connor',
 'Kyle Okposo',
 'Kyle Palmieri',
 'Lane Pederson',
 'Lars Eller',
 'Lassi Thomson',
 'Laurent Dauphin',
 'Lawrence Pilut',
 'Lawson Crouse',
 'Leon Draisaitl',
 'Liam Foudy',
 "Liam O'Brien",
 'Lias Andersson',
 'Libor Hajek',
 'Logan Brown',
 'Logan Couture',
 "Logan 

In [12]:
import plotly.graph_objects as go
import numpy as np
from moepy import lowess
from collections import namedtuple
import pickle
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor
from scipy.stats import norm
from datetime import datetime
import pandas as pd
import numpy as np
from numpy import random
from scipy.stats import norm
skaters = pd.read_csv(f'cleaned/master_skaters.csv')
skaters['exGF'] = skaters['TOI']*0.032+0.055
skaters['exGA'] = skaters['TOI']*0.032+0.055
skaters['eGF'] = skaters['TOI']*0.032+0.055
skaters['eGA'] = skaters['TOI']*0.032+0.055
skaters['GFAA'] = skaters['GF'] - skaters['eGF']
skaters['GAAA'] =  skaters['eGA'] - skaters['GA']
skaters['xGFAA'] = skaters['xGF'] - skaters['exGF']
skaters['xGAAA'] =  skaters['exGA'] - skaters['xGA']
skaters['Production'] = (0.75*skaters['Goals']) + (0.7*skaters['First Assists']) + (0.55*skaters['Second Assists']) + (0.5*skaters['ixG']) + (0.05*skaters['Shots Blocked']) 
skaters['Misc'] = (0.15*(skaters['Penalties Drawn'] - skaters['Total Penalties'])) + (0.01*(skaters['Faceoffs Won'] - skaters['Faceoffs Lost']))
skaters['Offense'] = np.where(skaters['Position'] == 'D', 
((0.425*(skaters['GF'] + skaters['GFAA'])) + (1.7*(skaters['xGF'] + skaters['xGFAA'])))/2.125 - 0.75,
(((0.625*(skaters['GF'] + skaters['GFAA'])) + (0.625*(skaters['xGF'] + skaters['xGFAA'])))/2) - 0.4)
skaters['Defense'] = np.where(skaters['Position'] == 'D', 
(((0.575*(-skaters['GA'] + skaters['GAAA'])) + (2.3*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 1.05, 
(((0.4375*(-skaters['GA'] + skaters['GAAA'])) + (1.7*(-skaters['xGA'] + skaters['xGAAA'])))/2) + 0.6)
skaters['Score'] = skaters['Production'] + skaters['Misc'] + skaters['Offense'] + skaters['Defense']
skaters = skaters.loc[:, ~skaters.columns.str.contains('^Unnamed')]
births = pd.read_csv('birthdates.csv')
merged = skaters.merge(births[['Player', 'Season', 'dob']], how='left', on=['Player', 'Season'])
merged.drop_duplicates(subset=['Player', 'Team', 'Date', 'Opposition', 'Season'], inplace=True)
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team == 'New York Islanders')),'1999-07-26', merged['dob'])
merged['dob'] = np.where(((merged.Player == 'Sebastian Aho') & (merged.Team != 'New York Islanders')),'1996-02-17', merged['dob'])
merged[["Date", "dob"]] = merged[["Date", "dob"]].apply(pd.to_datetime)
merged['Age'] = (merged["Date"] - merged["dob"]).dt.days/365
merged['RoundAge'] = np.floor(merged["Age"])


start_dates = {season: datetime.strptime((str(season)[:4] + '-10-04'), '%Y-%m-%d') for season in skaters.groupby(['Season']).count().index}
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: f'𝒩(μ={s[0]:.3f}, 𝜎²={s[1]:.3f})'
def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain

    x = x + K*y      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)

def get_projections(player, time_type):
    alpha = 0.99809326
    df = merged[merged.Player == player]
    df = df.sort_values(by='Date')
    df['Career Game'] = range(1, len(df) + 1)
    df['Adjusted'] = df['Score']*(50/1.55) + 50
    df[["Date", "dob"]] = df[["Date", "dob"]].apply(pd.to_datetime)
    projections = [0]
    scores = np.concatenate((np.zeros(10),  df['Score'].to_list()))
    total = 0

    for i in range(-9, len(df)):
        curr = 0

        j = i-1
        if j >= 0:

            weight = alpha**(df['Date'].to_list()[i] - df['Date'].to_list()[j]).days
        else:
            weight = alpha**(-1*j)
        
        curr += weight * projections[-1] * total
        curr += weight * scores[j+10]
        total += 1
        total *= weight
        projections.append((curr/total))            

        

    df['Projection'] = projections[10:]



    sensor_var = 1.3330171065772254
    process_var = 0.0013513089896074557
    process_model = gaussian(0., process_var)
    pos = gaussian(0., 100.)
    N = len(df) + 10
    zs = np.concatenate((np.zeros(10), df['Score'].to_list()))
    ps = []
    for i in range(N):
        prior = predict(pos, process_model)    
        pos = update(prior, gaussian(zs[i], sensor_var))
        ps.append(pos.mean)
    df['Kalman'] = ps[10:]

    # load the dataset
    series = df[['Projection', 'Kalman']]
    values = series.values

    # fit model
    model = pickle.load(open('finalized_model.sav', 'rb'))
    # construct an input for a new preduction
    preds = df['Kalman'].to_list()[:82]
    for i in range(82, len(df)):
        row = values[i-82:i].flatten()
    # make a one-step prediction
        yhat = model.predict(asarray([row]))
        preds.append(yhat[0])
    df['XGB'] = preds

    ages = range(18, 48)
    A = [-0.14396,-0.05957,0.021176,0.080782,0.144998,0.176731,
            0.181012,0.179595,0.162485,0.137085,0.114395,0.094328,
            0.071203,0.043198,0.013411,0.002706,-0.01087,-0.030415,
            -0.04996,-0.03732,-0.06303,-0.03521,-0.038033904,-0.033721952,
            -0.02941,-0.04036,-0.05131,-0.08806,-0.08892,-0.08978]
    poly  = np.polyfit(ages, A, 4)
    for i in range(1, len(preds)):
        prev_age = df['Age'].to_list()[i-1]
        prev_expected = np.polyval(poly, prev_age)
        curr_expected = np.polyval(poly, df['Age'].to_list()[i])
        multiplier = curr_expected/prev_expected
        if preds[i] < 0:
            preds[i] = preds[i]/multiplier
        else:
            preds[i] = preds[i] * multiplier



    for i in range(0, len(df)):
        season = df['Season'].to_list()[i]
        since_start = (df['Date'].to_list()[i] - start_dates[season]).days
        season_adjust = (-0.00109 * since_start  + 1.1)
        if df['Venue'].to_list()[i] == 'Home':
            preds[i] = preds[i]*1.35*season_adjust*1.2
        else:
            preds[i] = preds[i]*1.35*season_adjust*0.8


        prev = merged[(merged.Season == season) & (merged.Date < df['Date'].to_list()[i])]
        opps = prev.groupby(['Opposition']).mean(numeric_only=True)[['Score']]
        opps['Multiplier'] = (opps['Score']+5)/5.438829866884756
        mult = opps[opps.index == (df['Opposition'].to_list()[i])]['Multiplier']
        if len(mult) > 0:
            mult = mult[0]
        else:
            mult = 1
        preds[i] = preds[i] * mult

    df['adjXGB'] = preds
      
    # Model fitting
    frac = 0.25
    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['XGB']), frac=frac, num_fits=20, robust_iters=3)
    df['XGB_Lowess'] = df_quantiles[0.5].to_list()

    quant_reg_func = lowess.calc_quant_reg_betas
    quantile_model = lowess.Lowess(reg_func=quant_reg_func)
    quantile_model.fit(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, q=0.5, robust_iters=3)
    df_quantiles = lowess.quantile_model(np.asarray(df[time_type]), np.asarray(df['adjXGB']), frac=frac, num_fits=20, robust_iters=3)
    df['adjXGB_Lowess'] = df_quantiles[0.5].to_list()
    pots = []
    mults = []
    for i in range(0, len(df)):
        curr_age = df['Age'].to_list()[i]
        if curr_age <= 24.26795064:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = (0.37233423)/(curr_expected+0.2)
            mults.append(multiplier)
            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)

        elif curr_age <= 38.62152643:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
        elif curr_age <= 42.25458411:
            curr_expected = np.polyval(poly, curr_age)
            multiplier = 0.24220954/(curr_expected+0.2)
            mults.append(multiplier)

            if df['XGB_Lowess'].to_list()[i] < 0:
                pots.append(df['XGB_Lowess'].to_list()[i]/multiplier)
            else:
                pots.append(df['XGB_Lowess'].to_list()[i]  * multiplier)        
        else:
            mults.append(1)

            pots.append(df['XGB_Lowess'].to_list()[i] )
    df['multiplier'] = mults
    df['Potential'] = pots
    df.reset_index(inplace=True, drop=True)
    df_quantiles.reset_index(inplace=True, drop=True)
    return df, df_quantiles



In [14]:
players = list(merged[merged.Season == 20222023].Player.unique())
players.sort()
recents = []
errors = []
for player in players[600:]:
    print(player)
    try:
        df = pd.read_csv('projections/20222023/'+(player.replace(' ', '_'))+'.csv')
    except:
        df, df_quantiles = get_projections(player, 'Career Game')
        df.to_csv('projections/20222023/'+(player.replace(' ', '_'))+'.csv')
        recents.append(df[df.index == len(df)-1])

Milan Lucic


100%|██████████| 9/9 [00:05<00:00,  1.61it/s]


Miles Wood


100%|██████████| 9/9 [00:03<00:00,  2.36it/s]


Milos Kelemen


...

100%|██████████| 9/9 [00:04<00:00,  2.10it/s]


Zack MacEwen


100%|██████████| 9/9 [00:03<00:00,  2.47it/s]


Zemgus Girgensons


100%|██████████| 9/9 [00:03<00:00,  2.63it/s]


In [53]:
pd.read_csv('cleaned/master_skaters.csv')['Player'].drop_duplicates().sort_values()

386846           A.J. Greer
2644           Aaron Downey
290572         Aaron Ekblad
91152          Aaron Gagnon
3148          Aaron Johnson
                ...        
2517          Zack Stortini
4855        Zbynek Michalek
683             Zdeno Chara
246170    Zemgus Girgensons
42933         Zenon Konopka
Name: Player, Length: 2840, dtype: object

In [50]:
fig = go.Figure()
i = 0
colors = ['rgba(204, 25, 25', 'rgba(25, 114, 204', 'rgba(25, 204, 114', ]
for player in ['jalen_chatfield']:
    df = pd.read_csv(f'projections/20222023/{player}.csv')

    time_type = 'Career Game'
    fig.add_trace(go.Scatter(x=df[time_type], y=df['Projection'], mode='markers', opacity=0.15, marker=dict(color=colors[i] + ', 1)')))
    fig.add_trace(go.Scatter(x=df[time_type], y=df['XGB'], name=player, mode='markers', marker=dict(color=colors[i] + ', 1)'))) 
    # fill to trace0 y
    fig.add_trace(go.Scatter(x=df[time_type], y=df['XGB_Lowess'], name=player, mode='lines', marker=dict(color=colors[i] + ', 1)'))) # fill to trace0 y
    i += 1
fig.update_layout(autosize=False, width=1600, height=500)
fig.update_yaxes(range = [-1,3])
thresh = min(2.425, norm.ppf(0.98, loc=df['Potential'].mean(), scale=df['Potential'].std()))
recent = df['Potential'].to_list()[-1]
fig.show()
print(round(100*(df['XGB_Lowess'].to_list()[-1]+5)/(7.5)))
blend = min(2.425, (1.5*thresh+0.5*recent)/2)
print(round(100*(blend+5)/(7.5)))


89
89
